# Part 3: Other options for input

Umami is a package for calculating metrics for use with for Earth surface dynamics models. This notebook is the final notebook in a three-part introduction to using umami.

Umami was designed to work well with the [terrainbento](https://terrainbento.readthedocs.io/en/latest/) model package, as well as other models built using the [Landlab Toolkit](https://github.com/landlab/landlab). However, umami can be used with models built with other modeling tools and data in a variety of formats. 

## Scope of this tutorial

In this tutorial you will learn how to use other input options for umami. 

Specifically we will use square gridded terrain stored as a [netCDF](https://www.unidata.ucar.edu/software/netcdf/).

If you have comments or questions about the notebooks, the best place to get help is through [GitHub Issues](https://github.com/TerrainBento/umami/issues).

To begin this example, we will import the required python packages. 

In [ ]:
import numpy as np
from scipy.interpolate import interp2d

from urllib.request import urlopen
from io import BytesIO, StringIO

from landlab import imshow_grid, RasterModelGrid, VoronoiDelaunayGrid
from landlab.io import read_esri_ascii

from umami import Metric, Residual

Umami does not make any requirements regarding where terrain data comes from or what model or modeling package is used to construct modeled terrain. However, umami does require that modeled or observed terrain is provided to it as a Landlab grid with an at-node field called `topographic__elevation`. 

This requirement means that umami can take advantage of Landlab's [existing functions for input](https://landlab.readthedocs.io/en/release/landlab.io.html). Additionally, a Landlab grid can take a field based on a [numpy array](https://docs.scipy.org/doc/numpy/reference/generated/numpy.array.html). Thus any file format you can read into python as a numpy array can be used as a landlab grid. 

## Step 1: Read in and use a numpy array

In this example we will download data from the [OpenTopography REST server](https://opentopography.org/developers). We will download a small patch of land near Boulder, CO. You can change the values of `north`, `south`, `east` and `west` to change the location. 

These data are provided with horizontal units of degrees. For this example we will ignore 

In [ ]:
west=-105.35 # longitude (degrees)
east=-105.15 # longitude (degrees)
north=40.1 # latitude (degrees)
south=39.9 # latitue (degrees)

URL = "http://opentopo.sdsc.edu/otr/getdem?demtype=SRTMGL3&"
url = (URL + 
           "west=" + str(west) + "&" +
           "south=" + str(south) + "&"
           "east=" + str(east) + "&"
           "north=" + str(north) + "&"
           "outputFormat=AAIGrid")

f = urlopen(url)
b = BytesIO(f.read())
file_like = StringIO(b.getvalue().decode("UTF-8"))

We can use Landlab IO functions to read.
    
    
You could do this with gdal, or rasterio, or whatever you like... at the end of the day, we just care about z being a numpy array...

In [ ]:
mg, z = read_esri_ascii(file_like, name="topographic__elevation")
imshow_grid(mg, z)

z is just a numpy array. We can create a new grid and add a random numpy field to it. 

But Umami will care that the grids are the same. 

In [ ]:
mg.shape

In [ ]:
?read_esri_ascii

In [ ]:
mg.dx

In [ ]:
mg.xy_of_lower_left

In [ ]:
new_mg = RasterModelGrid(mg.shape, mg.dx, xy_of_lower_left=mg.xy_of_lower_left)
new_z = new_mg.add_field("node", "topographic__elevation", z + np.random.randn(z.size))
imshow_grid(new_mg, new_z)

In [ ]:
imshow_grid(new_mg, new_z-z)

In [ ]:
residuals = {
    "me": {
        "_func": "aggregate",
        "method": "mean",
        "field": "topographic__elevation"
    },
    "ep10": {
        "_func": "aggregate",
        "method": "percentile",
        "field": "topographic__elevation",
        "q": 10
    }
}
residual = Residual(new_mg, mg, residuals=residuals)
residual.calculate()

In [ ]:
residual.names

In [ ]:
residual.values

## Step 2: Use irregular data


In [ ]:
interp_obj = interp2d(mg.x_of_node, mg.y_of_node, z)

random_x = np.random.uniform(low=mg.x_of_node.min(), high=mg.x_of_node.max(), size=mg.x_of_node.size)
random_y = np.random.uniform(low=mg.y_of_node.min(), high=mg.y_of_node.max(), size=mg.y_of_node.size)

interp_z = interp_obj(random_x, random_z)

# going to need to re-order so that there isn't an issue. Could also just nudge each xy point around a little. 

In [ ]:
vdg = VoronoiDelaunayGrid(random_x, random_y)
vdg.add_field("node", "topographic__elevation", interp_z)

In [ ]:
metrics = {
    "me": {
        "_func": "aggregate",
        "method": "mean",
        "field": "topographic__elevation"
    },
    "ep10": {
        "_func": "aggregate",
        "method": "percentile",
        "field": "topographic__elevation",
        "q": 10
    }
}

metric = Metric(vdg, metrics=metrics)
metric.calculate()

In [ ]:
metric.names

In [ ]:
metric.values

# Next steps

Now that you have a sense for how the `Metric` and `Residual` classes are used, try the next notebook: [Part 4: Example application](ExampleApplication.ipynb).